In [1]:
from pathlib import Path

import automech
from project_utilities import p_, util, workflow
from protomech import mess

root_path = Path("..")
tag = "Z_combined_v0"

mech0 = automech.io.read(p_.stereo_mechanism(tag, "json", p_.data(root_path)))

In [2]:
mech0, _ = automech.replace_unstable_products(mech0)

/home/avcopan/proj/project-cyclopentane/code/protomech/src/automech/_mech.py:1279: UserWarning: This function ignores stereochemistry.
  warnings.warn("This function ignores stereochemistry.", stacklevel=1)


In [3]:
# mech = automech.merge_resonant_instabilities(mech0, remove=True)
# instab_mech = automech.resonant_instability_reactions(mech0)

In [4]:
# automech.resonant_unstable_species_names(mech0)

In [5]:
subpes_lst = ["01_01", "test"]
# subpes_lst = ["test"]
inp_name = "mess.inp"
mess0_path = p_.mess_source(root_path)
# mess.io.split_input(mess0_path / "example" / inp_name)
surfs = [mess.surf.from_mess_input(mess0_path / s / inp_name) for s in subpes_lst]
surf = mess.surf.combine(surfs)
# mess.net.display(surf, mech=mech0)

In [6]:
surf = mess.surf.merge_nmol_node_resonant_instabilities(surf, mech0)
mess.net.display(surf, mech=mech0)

In [9]:
subpes = "test"

mess_path = p_.mess(root_path)
out_dir = mess_path / subpes
out_dir.mkdir(exist_ok=True)
print(out_dir.resolve())

inp_file = out_dir / inp_name
inp_file.write_text(mess.surf.mess_input(surf))

/home/avcopan/proj/project-cyclopentane/data/mess/test


34182

In [7]:
# subpes = "test"

# mess_path = p_.mess(root_path)
# out_dir = mess_path / subpes
# out_dir.mkdir(exist_ok=True)
# print(out_dir.resolve())

# inp_file = out_dir / inp_name
# inp_file.write_text(mess.surf.mess_input(surf))